In [44]:
!pip install ipywidgets
!pip install flask ngrok
!pip install pyngrok
!pip install flask-cors




In [45]:
from google.colab import userdata, files
import json
import io
import torch
import ipywidgets as widgets
from IPython.display import display
from flask import Flask, request, jsonify
from pyngrok import ngrok

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import io
import json

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

def analyze_ipo(uploaded_file):
    try:
        file_content = io.BytesIO(uploaded_file.read())
        ipo_data = json.load(file_content)

        # --- Financial Calculations ---
        listed_price = float(ipo_data["Offer Details"].get("ListedPrice", 0))
        revenue_fy24 = float(ipo_data["Financial Data"].get("RevenueFromOperations2024", 0)) * 2

        try:
            net_worth_2024 = float(ipo_data["Financial Data"].get("NetWorth2024", 0))
            total_borrowings_2024 = float(ipo_data["Financial Data"].get("TotalBorrowings2024", 0))
            debt_to_equity = total_borrowings_2024 / net_worth_2024 if net_worth_2024 else "N/A"
        except (ZeroDivisionError, TypeError):
            debt_to_equity = "N/A"

        try:
            eps_2024 = float(ipo_data["Financial Data"].get("EarningsPerShare2024", 0))
            shares_outstanding = float(ipo_data["Financial Data"].get("SharesOutstandingPostIPO", 0))
            roe = (eps_2024 / (net_worth_2024 / shares_outstanding)) * 100 if eps_2024 and shares_outstanding else "N/A"
        except (ZeroDivisionError, TypeError):
            roe = "N/A"

        try:
            number_of_shares_offered = int(ipo_data["Offer Details"].get("NumberOfSharesOffered", 0))
            offer_price = float(ipo_data["Offer Details"].get("OfferPrice", 0))
            pe_ratio = offer_price / eps_2024 if eps_2024 else "N/A"
        except (ZeroDivisionError, TypeError, ValueError, AttributeError, KeyError):
            pe_ratio = "N/A"

        qualitative_factors = []
        risk_factors = ipo_data.get("Risk Factors", {})
        for risk_type in [
            "CompetitionRisk", "ClientConcentrationRisk", "DebtRisk",
            "EconomicGeopoliticalRisk", "SectorChangeRisk",
            "CustomerDependencyRisk", "TimePeriodRisk", "NegativeCashFlowRisk"
        ]:
            if risk_factors.get(risk_type):
                qualitative_factors.append(risk_factors[risk_type])
        qualitative_summary = ". ".join(qualitative_factors)

        try:
            listing_premium_discount = ((listed_price - offer_price) / offer_price) * 100 if offer_price else "N/A"
        except TypeError:
            listing_premium_discount = "N/A"

        risk_index = "High" if debt_to_equity > 1.5 else "Medium" if debt_to_equity > 1 else "Low"

        return {
            "OfferPrice": offer_price if offer_price else "N/A",
            "ListedPrice": listed_price,
            "RevenueFromOperationsFY24": revenue_fy24,
            "DebtToEquityFY24": debt_to_equity,
            "ROEFY24": roe,
            "PE": pe_ratio,
            "QualitativeFactors": qualitative_summary,
            "ListingPremiumDiscount": listing_premium_discount,
            "ShortTermOutlook": "N/A",
            "LongTermOutlook": "Uncertain",
            "RiskIndex": risk_index
        }

    except (json.JSONDecodeError, KeyError, TypeError, ValueError) as e:
        return {"error": f"Error processing file: {e}"}

@app.route('/analyze', methods=['POST'])
def analyze_ipo_api():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        # Log a confirmation for debugging
        print(f"File received: {file.filename}")

        output = analyze_ipo(file)
        return jsonify(output)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Open a ngrok tunnel to the Flask app
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> http://127.0.0.1:5000")
    app.run(debug=True, port=5000)


 * ngrok tunnel "NgrokTunnel: "https://8e6e-34-19-99-16.ngrok-free.app" -> "http://localhost:5000"" -> http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [38]:
# File Upload Widget and Handling
upload_button = widgets.FileUpload(accept=".json", multiple=False)
display(upload_button)

def handle_upload(change):
    # Access the first uploaded file (if any) using .keys() and list conversion
    uploaded_filenames = list(upload_button.value.keys())
    if uploaded_filenames:
        uploaded_filename = uploaded_filenames[0]  # Get the first filename
        uploaded_file = upload_button.value[uploaded_filename]
        # Get the file content from the 'content' key within the uploaded_file dictionary
        file_content = io.BytesIO(uploaded_file['content'])
        output = analyze_ipo(file_content) # Pass the file content, not the dictionary
        print(output)

upload_button.observe(handle_upload, names="value")

FileUpload(value={}, accept='.json', description='Upload')